<a href="https://colab.research.google.com/github/KasraOstad/Vineyard-Classification-in-Sussex/blob/master/post_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Similar to the GEE notebook, this notebook delved in extracting the relevant data for the 750 random vineyard points so that a 95% confidence interval of the median could be bootstrapped for each satellite band

In [ ]:
import ee
import time

#Authenticate to the Earth Engine servers
ee.Authenticate()

#Initialize the API
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=X8NuvKhUDO9x9fzKrdL9hUj__DELePAppwJi_hYvIMY&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/3AGpcgEK47WIMea96UE6zsul7DiI2UBXiFBQgB6ksx-t90YFQ9AQLQ4

Successfully saved authorization token.


In [ ]:
#Load vineyard random points
vineyardsID='users/aghajaniwork/random_points'
vineyard_fc= ee.FeatureCollection(vineyardsID)


vineyard_fc.size().getInfo()

750

In [ ]:
#Load Satellite of interest
collection=('COPERNICUS/S2_SR')
s2=ee.ImageCollection(collection)

#Define Area of interest
area_list= list ([(-0.831,51.1),(-0.831,50.711),(0.127,50.711),(0.127,51.1),(-0.831,51.1)])
AOI=ee.Geometry.Polygon(area_list)

#Filter image collection with AOI
s2=s2.filterBounds(AOI)

#Filter image collection with date
s2=s2.filterDate('2019-08-24','2019-08-26')

s2_list=s2.toList(s2.size())


#Find dates of our filtered image collection
dates=[]

for i in range(0, s2_list.size().getInfo()):
  image=ee.Image(s2_list.get(i))
  date=ee.Number.parse(image.date().format("YYYYMMdd"))
  dates.append(date.getInfo())

print(dates)  
s2.size().getInfo()

[20190825, 20190825]


2

In [ ]:
mosaic = s2.mosaic()

ndvi = mosaic.normalizedDifference(['B8', 'B4'])

ndwi=mosaic.normalizedDifference(['B8','B11'])

evi2 = mosaic.expression(
    '2.5 * ((NIR - RED) / (NIR + 2.4 * RED + 1))', {
    'NIR': mosaic.select('B8'),
    'RED': mosaic.select('B4')}
)

cl=mosaic.expression(
    '(RED3 / RED1) -1',{
    'RED3': mosaic.select('B7'),
    'RED1': mosaic.select('B5')
    }
)

#Create gg.image with all vegetation bands
veg=ndvi.addBands(ndwi)
veg=veg.addBands(evi2)
veg=veg.addBands(cl)

#Rename Vegetation bands
veg=veg.select(veg.bandNames().getInfo(),['NDVI', 'NDWI', 'EVI2', 'CL'])

s2_veg=mosaic.select(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12'])
s2_veg=s2_veg.addBands(veg)

print(s2_veg.bandNames().getInfo())


['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'NDVI', 'NDWI', 'EVI2', 'CL']


In [ ]:
#Filter Sentine 1 for our date and region of interest
sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD')\
              .filterDate('2019-08-25','2019-08-26')
sentinel1=sentinel1.filterBounds(AOI)

# Filter by metadata properties.
# 1. Filter to get images with VV and VH dual polarization.
# 2. Filter to get images collected in interferometric wide swath mode.
#IW, with VV+VH polarisation over land

vh = sentinel1.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
              .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
              .filter(ee.Filter.eq('instrumentMode', 'IW'))

    
vh_list=vh.toList(vh.size())

dates=[]

for i in range(0, vh_list.size().getInfo()):
  image=ee.Image(vh_list.get(i))
  date=ee.Number.parse(image.date().format("YYYYMMdd"))
  dates.append(date.getInfo())

print(dates)  


[20190825]


In [ ]:
s1=vh.first()


s1=s1.select(['VV','VH'])

s1_s2_veg=s2_veg.addBands(s1)
s1_s2_veg.bandNames().getInfo()


['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B8A',
 'B9',
 'B11',
 'B12',
 'NDVI',
 'NDWI',
 'EVI2',
 'CL',
 'VV',
 'VH']

In [ ]:
def s1_find_date (start,end):
  sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD')\
              .filterDate(start,end)
  sentinel1=sentinel1.filterBounds(AOI)

  # Filter by metadata properties.
  # 1. Filter to get images with VV and VH dual polarization.
  # 2. Filter to get images collected in interferometric wide swath mode.
  #IW, with VV+VH polarisation over land

  vh = sentinel1.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
                .filter(ee.Filter.eq('instrumentMode', 'IW')) 

  vh_list=vh.toList(vh.size())

  dates=[]

  for i in range(0, vh_list.size().getInfo()):
    image=ee.Image(vh_list.get(i))
    date=ee.Number.parse(image.date().format("YYYYMMdd"))
    dates.append(date.getInfo())
  
  print(dates)
  return vh_list

s1_im_list=s1_find_date('2019-04-29','2019-05-01')
s1_im2= ee.Image(s1_im_list.get(2))
s1_2=s1_im2.select(['VV','VH'])

s1_im_list=s1_find_date('2019-06-22','2019-06-24')
s1_im3= ee.Image(s1_im_list.get(0))
s1_3=s1_im3.select(['VV','VH'])

s1TS_s2_veg=s1_s2_veg.addBands(s1_2)
s1TS_s2_veg=s1TS_s2_veg.addBands(s1_3)
s1TS_s2_veg.bandNames().getInfo()


[20190429, 20190429, 20190430]
[20190623, 20190622]


['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B8A',
 'B9',
 'B11',
 'B12',
 'NDVI',
 'NDWI',
 'EVI2',
 'CL',
 'VV',
 'VH',
 'VV_1',
 'VH_1',
 'VV_2',
 'VH_2']

In [ ]:
#Extract satellite band information for all our polygons
#Restricted to only 3 bands at a time
def feature_extract(fc,img,fol):
  asset_name='users/aghajaniwork/'+fc
  polygons= ee.FeatureCollection(asset_name)

  polygon_stats=img.reduceRegions(**{
    'collection':polygons,
    'reducer':ee.Reducer.median(),
    'scale':  10 
  })

  task=ee.batch.Export.table.toDrive(**{
  'collection': polygon_stats,
  'description': fc,
  'fileFormat': 'SHP',
  'folder':fol
  })

  task.start()

  while task.active():
    print('Extraction Processing...'.format(task.id))
    time.sleep(30)

  return  task.status() 



In [ ]:
feature_extract('random_points',s1TS_s2_veg,'vin_rp2')

Extraction Processing...
Extraction Processing...


{'creation_timestamp_ms': 1597191002701,
 'description': 'random_points',
 'destination_uris': ['https://drive.google.com/#folders/1qSbxt37AWfxC_n5vr4V6jAlAWx8s3-m8',
  'https://drive.google.com/#folders/1qSbxt37AWfxC_n5vr4V6jAlAWx8s3-m8',
  'https://drive.google.com/#folders/1qSbxt37AWfxC_n5vr4V6jAlAWx8s3-m8',
  'https://drive.google.com/#folders/1qSbxt37AWfxC_n5vr4V6jAlAWx8s3-m8',
  'https://drive.google.com/#folders/1qSbxt37AWfxC_n5vr4V6jAlAWx8s3-m8',
  'https://drive.google.com/#folders/1qSbxt37AWfxC_n5vr4V6jAlAWx8s3-m8'],
 'id': '6WOJPJJD2CPFMEGHNYMAG53V',
 'name': 'projects/earthengine-legacy/operations/6WOJPJJD2CPFMEGHNYMAG53V',
 'start_timestamp_ms': 1597191022408,
 'state': 'COMPLETED',
 'task_type': 'EXPORT_FEATURES',
 'update_timestamp_ms': 1597191034504}